#### **Simple Command Recognition Using Neural Networks**  
Tyler Sagardoy   
Udacity Machine Learning Nanodegree Capstone Project   
June 2018   

This report details my project to build an algorithm that can understand a small library of 10 simple commands while being able to ignore, or distinguish from, unknown commands. Using Mel-Frequency Capestral Coefficients to regularize and convey communicated information, I'll train a 1-dimensional convolutional neural network using RMSprop to minimize the categorical crossentropy log loss to process and interpret the information in order to make a prediction as to the intended command.

I begin by discussing trends in command recognition, followed by a review of my project objective, a description of the data used for model training, and an outline of my project. I proceed to discuss steps taken during data preprocessing, including a brief overview of Mel-Frequency Capestral Coefficients and how they are applied, before discussing the architectures of the 3 networks to be trained. Each network was trained against hyperparameters in dropout rates, learning rates, and leaky rectified linear unit activation kernel alphas resulting in 81 separately trained models. From these models, I select the one that scores best when applied against an exclusive set of testing data and build a simple speech recognition algorithm from it.

#### Simple Command Recognition, Rise & Market Trends

Within the past few years, voice command and activation has become a popular trend in user experience and design. Voice command allows for technologies to meet specific consumer demands, such as providing a user interface when hands or vision is occupied – a reason 61 percent of users state why they use voice command – or when one is in the car or on the go – the primary setting for voice usage in 55 percent of instances (Young, 2016). According to Google CEO Sundar Pichai, 20 percent of all queries made in 2016 on Google apps and Android devices were voice searches (Sterling, 2016).

The future looks bright for voice command. Comscore estimates that 50 percent of searches will be conducted by voice by 2020 (Young, 2016). While that figure may be highly optimistic, a study of 39 leading SEO experts identified voice usage as one of the top 3 important trends in search (Alameda Internet Marketing, 2016). While these estimates speak to the Internet search industry exclusively, the enthusiasm for voice activation in this industry portends demand for adoption in other industries.

As independent makers and entrepreneurs develop their products to feature this new and exciting interface method, or learn about it for the first time, many find that their needs for voice commands are minimal – only limited to a handful of basic operational commands. Many robust voice recognition algorithms are extraneous, costly, and unnecessary for many projects. Therefore, an opportunity exists to develop an open-source speech recognition algorithm intended for smaller developers and entry-level enthusiasts.

#### Project Purpose, Objectives & Objectives

The purpose of this report is to provide an algorithm that understands a small selection of simple audio commands. The selection of commands the algorithm should understand should be “Yes,” “No”, “Up”, “Down”, “Left”, “Right”, “On”, “Off”, “Stop” and “Go”.

I selected this project originally to compete in the TensorFlow Speech Recognition Challenge hosted by Kaggle and sponsored by Google; however, I used the data, guidance and competition objectives to outline my own goals and I narrowed the scope of my project to accomplish my primary objective - training a simple 1-D convolutional neural network to accurately predict with at least 80% accuracy the library of ten words. For words and sounds that the algorithm isn't explicitly trained on, the algorithm will be trained to classify in a catch-all 'other' category.

Some of the principles I incorporated into my project include:
- Use the standard 16-bit [16000,1] integer tensor format for audio PCM-decoded at 16000 Mbps as an algorithm input
- Output will be a [1, 11] tensor representing the prediction of the algorithm. Values will be between 0 and 1
- Regularize decoded audio into tensors of Mel-Frequency Cepstral Coefficients (MFCCs)
- Objectively work to minimize the number of layers in the architecture to maintain some simplicity within the model and to minimize runtime
- Use convolutional layers with a wide convolution window and moderate stride to identify speech patterns within inputs
- Use pooling layers to reduce dimensionality between convolutional layers
- Hyperparameters that I changed with each model were the dropbout rate, the learning rate, and the alpha of the rectified linear unit kernels (0,-0.1, 1)

#### Dataset & Inputs

The algorithm will be trained and tested using the Speech Commands Dataset released by Google on August 3, 2017. The data contains 64,727 one-second audio clips of 30 short words. The audio files were crowdsourced by Google with the goal of collecting single-word commands (rather than words as said and used in conversation). A group of 20 core words were recorded with most speakers saying them 5 times. An additional group 10 words were recorded to help distinguish unrecognized words; most speakers recorded these words once. The core words consist of “Yes,” “No”, “Up”, “Down”, “Left”, “Right”, “On”, “Off”, “Stop”, “Go” and the numbers zero through nine. Auxiliary words consist of "Bed", "Bird", "Cat", "Dog", "Happy", "House", "Marvin", "Sheila", "Tree" and "Wow". 

This algorithm will take as inputs
•	An file of PCM-encoded data to be decoded into a 16-bit [16000, 1] integer tensor
•   Output will be a [1, 11] tensor representing the prediction of the algorithm. Values will be between 0 and 1.

#### Comparative Benchmark

With 10 command labels and an additional ‘non-command’ label included in the library, a naïve algorithm that outputs only one label would be correct 1 out of 11 times, or 9.1% of the time. For purposes of assessing whether a CNN has any predictive power, I will use this low 9.1% threshold. However, for practical and usability purposes, I hope to train a CNN that has at least 80% accuracy – a threshold that would have put me within the top quartile of competitors, indicating that my model performed better than 75.0% of all other models.

#### Evaluation Metrics

Models will be evaluated using Multiclass Accuracy, defined as the ratio of correct classifications over total classifications. The logic behind this evaluation is rather straight-forward: out of the number of testing cases, how many did the algorithm respond correctly?

#### Project Outline

The procedural outline is as follows:
1.	Download the Speech Commands Dataset from Google and Kaggle.
2.	Decode the PCM-encoded audio files at 16000 Mbps into standardized 16-bit integer tensor of shape [16000, 1].
3.  Preprocess data into tensors of Mel-Frequency Cepstral Coefficients (MFCCs) 
4.	Divide dataset into training, testing, and validation sets with inputs and one-hot encoded targets.
5.	Define the architectures for the networks.
6.	Compile and train models, retaining only the models with the best classification accuracy on the validation set
7.	Use the test set to calculate Multiclass Accuracy and obtain at least 80.0% on one of the models trained.

#### 1. Download Speech Commands Dataset from Google and Kaggle

As described by Kaggle, the Speech Commands Dataset is a set of 64,727 1-second .wav audio files, each containing a single spoken English word. These words are from a small set of commands, and are spoken by a variety of different speakers. The audio files are organized into folders based on the word they contain, and this data set is designed to help train simple machine learning models. The dataset for this project was downloaded at https://www.kaggle.com/c/tensorflow-speech-recognition-challenge/data. Data was released by Google on August 3, 2017.

#### 2. Decode Audio Files

##### 2a. Download and Import Necessary Python Libraries

Before I begin decoding the Speech Command Dataset audio files, I need to download necessary libraries, modules and utilities into my programming environment, and import them into my iPython kernel. The libraries (and versions) I will be using are:
- SciPy (v0.19.1) - Python's scientific computing library; used in this project to directly decode PCM-encoded audio files at a rate of 16000 Mbps and to perform Discrete Cosine Transforms during MFCC calculations
- Numpy (v1.13.3) - library useful for handling and processing arrays of values and objects; used in this project to manage data and for performing Discrete Fourier Transforms during MFCC calculations
- Keras (v2.1.6) with TensorFlow (v1.8.0) backengine - Keras provides a high-level programming interface for deep learning and neural network development; Keras runs on top of TensorFlow, a library and engine for dataflow programming; used in this project to develop and train the neural networks and for one-hot encoding all target labels
- Glob - standard Python file searching library; used in this project to extrapolate filenames and paths for all audio files
- OS - standard Python operating system interface; used in this project to combine strings and values into valid file pathnames
- Math - standard Python mathematical functions; used in this project to calculate frequency bins in MFCC calculations

In [1]:
import os
from scipy.io import wavfile
from scipy.fftpack import dct
import numpy as np
import glob
import keras
from keras.utils import np_utils
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Conv1D, ZeroPadding1D, Dropout, AveragePooling1D

C:\Users\Tyler\Anaconda3\envs\tfspeech\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


##### 2b. Generate Lists of Training, Validation & Testing FIlenames

The dataset downloaded from Keras contained two presorted lists of audio filepaths for validation and training lists. For purposes of transparency and ease, I made a third list of training files by first creating a list of all audio filepaths, then removing any filepaths that were found in either the validation or training lists. The list was saved with the other two lists. 

In [5]:
# set directory path
dir_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'audio/audio')

# import pre-selected list of testing and validation audio filenames
testing_path = os.path.join(dir_path,'testing_list.txt')
testing_list = np.genfromtxt(testing_path, dtype=None, encoding=None)
print("Retrieved testing_list from "+ testing_path+ ". Size is " + str(testing_list.shape)+".")
validation_path = os.path.join(dir_path,'validation_list.txt')
validation_list = np.genfromtxt(validation_path, dtype=None, encoding=None)
print("Retrieved testing_list from "+ validation_path + ". Size is " + str(validation_list.shape)+".")

# generate exclusive list of training audio filenames
audio_path = os.path.join(dir_path, 'audio')
all_files_list = np.array(glob.glob(os.path.join(audio_path, '*', '*.wav')))
np.savetxt('audio/all_files_list.txt',all_files_list, encoding=None, fmt='%.100s')
audio_path_length = len(audio_path)+1

# create training_list by removing paths if in testing_list or validation_list
training_list = np.array([])
for index, value in np.ndenumerate(all_files_list):
    value = value[audio_path_length:]
    all_files_list[index] = value
    if np.isin([value],testing_list)==False and np.isin([value],validation_list)==False:
        training_list = np.append(training_list, all_files_list[index])
print(str(training_list.shape) + " is the shape of the training_list.")
np.savetxt('audio/training_list.txt',training_list, encoding=None, fmt='%.100s')

Retrieved testing_list from C:\Users\Tyler\Desktop\Capstone WIP\audio\testing_list.txt. Size is (6835,).
Retrieved testing_list from C:\Users\Tyler\Desktop\Capstone WIP\audio\validation_list.txt. Size is (6798,).
(51088,) is the shape of the training_list.


#### 3. Calculating Mel-Frequency Cepstral Coefficients (MFCCs)

This project owes immense gratitude to Practical Cryptography for their excellent tutorial on Mel Frequency Cepstral Coefficients (MFCCs). MFCCs are a widely used tool in Speech Recognition and the logic behind them is fairly simple: human beings communicate by producing sounds through manipulation of their vocal tract (e.g. shape of tongue, position of tongue near teeth, shape of lips, etc.). That is, sound carries information about what is physically being communicated. In short durations, this information is manifested across ranges in the audio power spectrum. MFCCs describe the shape of this information.

To calculate MFCCs for each audio file, I first split the audio files into 2.5ms frames of 400 samples each. For this analysis, I added a stride of 200 samples - thus resulting in 79 frames total. For each frame, I performed a Discrete Fourier Transform to convert the signal from the time domain to the frequency domain. Since DFTs return complex numbers signifying both phase and amplitude, and because I'm only interested in the amplitudes of frequencies (and therefore real numbers), I calculate the periodogram estimate of the power spectrum by squaring each frequency power and dividing the result by the number of samples in the frame size. Only the first half of frequencies are kept; the rest are dropped.

Signals are now converted from frequency to Mels, which relates perceived pitch of a sound to its actual measured frequency. We aggregate these signals by frequency into overlaping bins of triangular filters to measure broader definitions of pitch. These bins span the reach of the human vocal spectrum (300hz to 8000hz). In typical MFCC calculations, between 20 and 40 filters are used in the filterbank. For this project, I use 26. The log of the summation of the energies in each filterbank is taken. We are left with a waveform described by 26 log filterbank energies. This waveform is decomposed using a Discrete Cosine Transform; the results are the MFCCs, describing the shape of the wavelength the power of pitches within the frame. In this analysis, only coefficients 2 - 13 are kept. In practice, typically the first half of coefficients are kept. 

Once all frames are compiled, we are left with an array of 79 overlapping frames, each with 12 MFCCs. 

In [2]:
# given a filename, mfcc_conversion returns mel frequency cepstral coefficients array
# mfcc_conversion returns array of (79,12) representing 79 audio frames described by 12 coefficients
def mfcc_conversion(audio_path, filename, sample_rate = 16000, frame_size = 400, stride_size = 200, nfft = 512):
    # decode audio
    decoded_audio = audio_decoder(str(os.path.join(audio_path, filename)))    
    audio = decoded_audio.reshape((16000,1))
    
    first_index = 0
    mfcc_coefficients = np.empty((0,12))
    # apply the following for each signal frame
    while first_index <= audio.shape[0]-frame_size:
        last_index = first_index+frame_size
        frame = audio[first_index:last_index,:]
        
        # calculate discrete Fourier transform
        frame = np.fft.fft(frame, n = nfft, axis=0)
        
        # calculate the periodogram estimate of the power spectrum; drop last half of values
        power_spectrum = np.absolute(np.square(frame))/frame_size
        power_spectrum = power_spectrum[0:int(nfft/2),:].astype(float)
        power_spectrum = power_spectrum.reshape((power_spectrum.shape[0],))
        # print(power_spectrum)
        
        # apply the mel filterbank to the power spectra, sum the energy in each filter
            # frequencies on which to define mel filterbanks
        mel_freqs = np.array([300, 383.4, 473.8, 571.7, 677.8, 792.7, 917.3, 1052.2, 1198.3, 1356.7, 1528.3,
                              1714.2, 1915.6, 2133.7, 2370.1, 2626.3, 2903.7, 3204.4, 3530.1, 3882.9, 4265.2,
                              4679.4, 5128.2, 5614.4, 6141.1, 6711.8, 7330.1, 8000]).astype(float)
        vfunc = np.vectorize(bin_index)
        mel_bins = vfunc(mel_freqs, nfft=nfft, sample_rate=16000)
        # print(mel_bins)
        
            # calculate filterbank
        mel_filterbank = np.empty((26,256))
        for i in range(0,mel_filterbank.shape[0]):
            for j in range(0,mel_filterbank.shape[1]):
                mel_bin_min = mel_bins[i]
                mel_bin_mid = mel_bins[i+1]
                mel_bin_max = mel_bins[i+2]
                if j >= mel_bin_min and j < mel_bin_mid:
                    filter = float(j - mel_bin_min) / float(mel_bin_mid - mel_bin_min)
                elif j >= mel_bin_mid and j < mel_bin_max:
                    filter = float(mel_bin_max- j) / float(mel_bin_max - mel_bin_mid)
                else:
                    filter = float(0)
                mel_filterbank[i,j] = filter
                
                # apply filterbank to power spectra and calculate log filterbank energies
        logbankenergies = np.zeros((mel_filterbank.shape[0]))
        for i in range(0, mel_filterbank.shape[0]):
            mel_filters = mel_filterbank[i,:]
            bankenergy = np.dot(power_spectrum,mel_filters)+1
            logbankenergies[i] = np.log(bankenergy)
            
        
        # take the discrete cosine transform of the log filterbank energies
        log_dct = dct(logbankenergies)
        
        # saving DCT coefficients 2-13; discard rest
        log_dct = np.transpose(log_dct[1:13])
        i = first_index/stride_size
        mfcc_coefficients = np.vstack((mfcc_coefficients, log_dct))
                
        # set up next frame
        first_index = first_index + stride_size
    
    return mfcc_coefficients

# calculates bin index given frequency and sample_rate
def bin_index(frequency, nfft=512, sample_rate=16000):
    bin = math.floor((nfft+1)*frequency/sample_rate)
    return bin

# decodes audio given a file name
def audio_decoder(filename):
    rate, data = wavfile.read(filename)
    data = np.array(data)
    if data.shape[0]<=16000:
        difference = 16000 - data.shape[0]
        data = np.append(data,np.zeros((difference,)))
        data = np.transpose(data.reshape((16000,1)))
        data = data.astype(int)
    return data

#### 4. Prepare Data and One-Hot Encoded Targets

##### 4a. Calculating training, validation, and testing MFCC datasets

Each list of audio files is decoded into MFCC data arrays and saved to disk. 

In [ ]:
# initialize MFCC data arrays in memory
x_training = np.empty((0, 79, 12), float)
x_validation = np.empty((0, 79, 12), float)
x_testing = np.empty((0, 79, 12), float)

audio_path = str(os.path.join('audio','audio'))

# decode training_list
for i in range(0, training_list.shape[0]):
    mfccs = mfcc_conversion(training_list[i])
    mfccs = np.expand_dims(mfccs, axis=0)
    x_training = np.vstack((x_training, mfccs))
np.save('data/x_training', x_training)
print("Training list audio decode complete.")

# decode validation_list
for i in range(0, validation_list.shape[0]):
    mfccs = mfcc_conversion(validation_list[i])
    mfccs = np.expand_dims(mfccs, axis=0)
    x_validation = np.vstack((x_validation, mfccs))
np.save('data/x_validation', x_validation)    
print("Validation list audio decode complete.")

# decode testing_list
for i in range(0, testing_list.shape[0]):
    mfccs = mfcc_conversion(testing_list[i])
    mfccs = np.expand_dims(mfccs, axis=0)
    x_testing = np.vstack((x_testing, mfccs))
np.save('data/x_testing', x_testing)
print("Testing list audio decode complete.")

##### 4b. Categorizing Targets

The models will train on targets reflecting 1 of 11 types of classifications - the 10 words and an 'other' catchall category for unknown sounds. To get these target labels, I look at the parent folder of the audio files and classify as one of 11 numbers. I then use Keras to turn my integer labels into one-hot encoded vectors of size 11, and add a dimension at axis 1 in order for the model to process. I save the results to disk. 

In [6]:
# audio targets are determined by the name of their parent folder
def audio_categorizer(x_arr):
    y_arr=np.array([])
    for index, value in np.ndenumerate(x_arr):
        cat_name = os.path.dirname(value)
        if cat_name == "yes":
            y_arr = np.append(y_arr,1)
        elif cat_name == "no":
            y_arr = np.append(y_arr,2)
        elif cat_name == "up":
            y_arr = np.append(y_arr,3)
        elif cat_name == "down":
            y_arr = np.append(y_arr,4)
        elif cat_name == "left":
            y_arr = np.append(y_arr,5)
        elif cat_name == "right":
            y_arr = np.append(y_arr,6)
        elif cat_name == "on":
            y_arr = np.append(y_arr,7)
        elif cat_name == "off":
            y_arr = np.append(y_arr,8)
        elif cat_name == "stop":
            y_arr = np.append(y_arr,9)
        elif cat_name == "go":
            y_arr = np.append(y_arr,10)
        else:
            y_arr = np.append(y_arr,0)
    return y_arr 

# categorize training labels
y_training = audio_categorizer(training_list)
y_validation = audio_categorizer(validation_list)
y_testing = audio_categorizer(testing_list)

# applying a one-hot encoding scheme
y_training = np_utils.to_categorical(y_training, 11)
y_validation = np_utils.to_categorical(y_validation, 11)
y_testing = np_utils.to_categorical(y_testing, 11)

# add dimension for network processing
y_training = np.expand_dims(y_training, axis=1)
y_validation = np.expand_dims(y_validation, axis=1)
y_testing = np.expand_dims(y_testing, axis=1)

print('The shape of training targets is '+str(y_training.shape))
print('The shape of validation targets is '+str(y_validation.shape))
print('The shape of testing targets is '+str(y_testing.shape))

# save MFCC datasets for future use
np.save('data/y_training', y_training)
np.save('data/y_validation', y_validation)
np.save('data/y_testing', y_testing)


The shape of training targets is (51088, 1, 11)
The shape of validation targets is (6798, 1, 11)
The shape of testing targets is (6835, 1, 11)


#### 5. Model Architecture

For model training and evaluation, I developed three different model architectures representing three different types of reasoning. In developing these architectures, I sought to balance these (often-competing) principles: 
- Objectively work to minimize the number of layers in the architecture to maintain some simplicity within the model and to minimize runtime
- Use convolutional layers with a wide-enough convolution window and stride to identify patterns
- Use pooling layers to reduce dimensionality between convolutional layers

Each of the network uses 1-dimensional convolutional layers that slide along the time axis, convoluting MFCCs and reducing the size of the first dimension of the audio data. The idea behind each of these networks is to recognize patterns among MFCCs across time. Differentiating features among these three models were the size of the convolution windows, the overlapping stride of the windows, the number of filters, and the number of layers. The three networks I built are: 
1. Large Windows/Few Layers/Many Filters - this model consists of 1 padding layer, 2 convolutional layers with window size of 10 and stride of 5 each, an average pooling layer, and a dense output layer. The convolutional layers have 50 and 100 filters, respetively. Dropout layers have been added after each convolutional layer. 
2. Small Windows/Fewer Filters - this model consists of 5 sets of convolutional layers with window size of 3 and strides of 2 followed by a dense output layer. Each convolutional layer has 20 filters. Dropout layers have been added after each convolutional layer. 
3. Moderate Windows/Increasing Filters - this model consists of a set of 1 padding and 2 convolutional layers with window size 4 and stride 2, followed by another set of 1 padding and 3 convolutional layers with small window sizes. The number of filters increase with each additional layer until the final dense output layer. Dropout layers have been added after each convolutional layer. 

Hyperparameters that I changed with each model were the dropbout rate (0%,25%,50%), the learning rate (0.01, 0.05, 0.10), and the alpha of the rectified linear unit kernels (0,-0.1, -0.5). Therefore, the total number of models evaluated was 81. 


In [7]:
# Model_1 is the Large Windows/Many Filters network
def model_1_architecture(dropout_rate=0):
    model = Sequential()
    model.add(ZeroPadding1D(input_shape=(79, 12),padding=(0,1)))
    model.add(Conv1D(50, kernel_size=10, strides=5, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(100, kernel_size=10, strides=5, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(AveragePooling1D(pool_size=2))
    model.add(Dense(11, activation='softmax'))
    return model

# Model_2 is the Small Windows/Fewer Filters network
def model_2_architecture(dropout_rate=0):
    model = Sequential()
    model.add(Conv1D(20, kernel_size=3, strides=2, activation='relu', input_shape=(79, 12)))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(20, kernel_size=3, strides=2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(20, kernel_size=3, strides=2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(20, kernel_size=3, strides=2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(20, kernel_size=3, strides=2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(11, activation='softmax'))
    return model

# Model_3 is the Moderate Windows/Increasing Filters network
def model_3_architecture(dropout_rate=0):
    model = Sequential()
    model.add(ZeroPadding1D(input_shape=(79, 12),padding=(0,3)))
    model.add(Conv1D(10, kernel_size=4, strides=2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(20, kernel_size=4, strides=2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(ZeroPadding1D(padding=(0,1)))
    model.add(Conv1D(40, kernel_size=4, strides=2, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(60, kernel_size=3, strides=3, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Conv1D(80, kernel_size=3, strides=1, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(11, activation='softmax'))
    return model

model = model_1_architecture()
print("Model 1 Network Summary")
model.summary()

model = model_2_architecture()
print("Model 2 Network Summary")
model.summary()

print("Model 3 Network Summary")
model = model_3_architecture()
model.summary()

Model 1 Network Summary
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding1d_1 (ZeroPaddin (None, 80, 12)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 15, 50)            6050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 50)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 2, 100)            50100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 2, 100)            0         
_________________________________________________________________
average_pooling1d_1 (Average (None, 1, 100)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 11)          

#### 6. Model Training

For network training, I trained the model on random batches of 1000 files per epoch over 60 epochs to ensure that most, if not all, of my 51,100 training data is used. I used the RMSprop optimization algorithm to minimize categorical crossentropy log loss and validated my models comparing both log loss from training and validation. Only the models that minimize log loss are saved to disk. I also enabled early stopping to be triggered after 10 epochs of no improvement in log loss. 

Hyperparameters - dropbout rates, learning rates, and the leaky rectified linear unit alpha - were toggled for each of the 3 models resulting in 81 trained models.

In [15]:
def model_1_train(x_training, y_training,x_validation,y_validation, dropout_rate=0.0,
                  learning_rate=0.01, relu_alpha=0.0, batch_size=1000, epochs=60):
    model = model_1_architecture(dropout_rate)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    dir_path = os.path.join(os.path.dirname(os.path.abspath('__file__')),'models')
    file_name = 'model1_dr'+str(dropout_rate)+'_lr'+str(learning_rate)+'_ra'+str(relu_alpha)+'.hdf5'
    checkpoint_file = os.path.join(dir_path, file_name)
    checkpointer = ModelCheckpoint(filepath=checkpoint_file, verbose=1, save_best_only=True)
    early_stopper = EarlyStopping(patience=10, verbose=1)
    hist = model.fit(x_training, y_training, batch_size=batch_size, epochs=epochs,
                  validation_data=(x_validation, y_validation), callbacks=[checkpointer, early_stopper], 
                  verbose=2, shuffle=True)

def model_2_train(x_training, y_training,x_validation,y_validation,dropout_rate=0.0,
                  learning_rate=0.01, relu_alpha=0.0, batch_size=1000, epochs=60):
    model = model_2_architecture(dropout_rate)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    dir_path = os.path.join(os.path.dirname(os.path.abspath('__file__')),'models')
    file_name = 'model2_dr'+str(dropout_rate)+'_lr'+str(learning_rate)+'_ra'+str(relu_alpha)+'.hdf5'
    checkpoint_file = os.path.join(dir_path, file_name)
    checkpointer = ModelCheckpoint(filepath=checkpoint_file, verbose=1, save_best_only=True)
    early_stopper = EarlyStopping(patience=10, verbose=1)
    hist = model.fit(x_training, y_training, batch_size=batch_size, epochs=epochs,
                  validation_data=(x_validation, y_validation), callbacks=[checkpointer,early_stopper], 
                  verbose=2, shuffle=True)
    
def model_3_train(x_training, y_training,x_validation,y_validation,dropout_rate=0.0,
                  learning_rate=0.01, relu_alpha=0.0, batch_size=1000, epochs=60):
    model = model_3_architecture(dropout_rate)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    dir_path = os.path.join(os.path.dirname(os.path.abspath('__file__')),'models')
    file_name = 'model3_dr'+str(dropout_rate)+'_lr'+str(learning_rate)+'_ra'+str(relu_alpha)+'.hdf5'
    checkpoint_file = os.path.join(dir_path, file_name)
    checkpointer = ModelCheckpoint(filepath=checkpoint_file, verbose=1, save_best_only=True)
    early_stopper = EarlyStopping(patience=10, verbose=1)
    hist = model.fit(x_training, y_training, batch_size=batch_size, epochs=epochs,
                  validation_data=(x_validation, y_validation), callbacks=[checkpointer,early_stopper], 
                  verbose=2, shuffle=True)

# load training and validation data into memory
x_training = np.load('data/x_training.npy')
x_validation = np.load('data/x_validation.npy')

y_training = np.load('data/y_training.npy')
y_validation = np.load('data/y_validation.npy')

# hyperparameters
dropout_rates = [0.00,0.25,0.50]
learning_rates = [0.01,0.05,0.10]
relu_alphas = [0,-0.1,-0.5]

batch_size = 1000
epochs = 60

for dropout_rate in dropout_rates:
    for learning_rate in learning_rates:
        for relu_alpha in relu_alphas:
            model = model_1_train(x_training, y_training,x_validation,y_validation,
                                         dropout_rate=dropout_rate,learning_rate=learning_rate,
                                         relu_alpha=relu_alpha, batch_size=batch_size, epochs=epochs)
            model = model_2_train(x_training, y_training,x_validation,y_validation,
                                         dropout_rate=dropout_rate,learning_rate=learning_rate,
                                         relu_alpha=relu_alpha, batch_size=batch_size, epochs=epochs)
            model = model_3_train(x_training, y_training,x_validation,y_validation,
                                         dropout_rate=dropout_rate,learning_rate=learning_rate,
                                         relu_alpha=relu_alpha, batch_size=batch_size, epochs=epochs)
    

Train on 51088 samples, validate on 6798 samples
Epoch 1/60
 - 5s - loss: 2.7634 - acc: 0.5947 - val_loss: 1.1854 - val_acc: 0.6203

Epoch 00001: val_loss improved from inf to 1.18544, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.01_ra0.hdf5
Epoch 2/60
 - 4s - loss: 0.9945 - acc: 0.6972 - val_loss: 0.8687 - val_acc: 0.7242

Epoch 00002: val_loss improved from 1.18544 to 0.86875, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.01_ra0.hdf5
Epoch 3/60
 - 4s - loss: 0.7447 - acc: 0.7624 - val_loss: 0.8318 - val_acc: 0.7287

Epoch 00003: val_loss improved from 0.86875 to 0.83176, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.01_ra0.hdf5
Epoch 4/60
 - 4s - loss: 0.6096 - acc: 0.8050 - val_loss: 0.6863 - val_acc: 0.7823

Epoch 00004: val_loss improved from 0.83176 to 0.68626, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.01_ra0.hdf5
Epoch 5/60
 - 4s - loss: 0.5254 - acc: 0.83

 - 3s - loss: 0.6273 - acc: 0.8006 - val_loss: 0.7514 - val_acc: 0.7555

Epoch 00022: val_loss did not improve from 0.70684
Epoch 23/60
 - 3s - loss: 0.6198 - acc: 0.8010 - val_loss: 0.6996 - val_acc: 0.7727

Epoch 00023: val_loss improved from 0.70684 to 0.69962, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.0_lr0.01_ra0.hdf5
Epoch 24/60
 - 3s - loss: 0.6118 - acc: 0.8038 - val_loss: 0.6807 - val_acc: 0.7751

Epoch 00024: val_loss improved from 0.69962 to 0.68072, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.0_lr0.01_ra0.hdf5
Epoch 25/60
 - 3s - loss: 0.6000 - acc: 0.8083 - val_loss: 0.6591 - val_acc: 0.7871

Epoch 00025: val_loss improved from 0.68072 to 0.65908, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.0_lr0.01_ra0.hdf5
Epoch 26/60
 - 3s - loss: 0.5890 - acc: 0.8121 - val_loss: 0.7508 - val_acc: 0.7661

Epoch 00026: val_loss did not improve from 0.65908
Epoch 27/60
 - 3s - loss: 0.5860 - acc: 0.8125 - val_


Epoch 00010: val_loss improved from 0.65389 to 0.62003, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.0_lr0.01_ra0.hdf5
Epoch 11/60
 - 4s - loss: 0.4836 - acc: 0.8431 - val_loss: 0.6249 - val_acc: 0.8076

Epoch 00011: val_loss did not improve from 0.62003
Epoch 12/60
 - 4s - loss: 0.4648 - acc: 0.8463 - val_loss: 0.5991 - val_acc: 0.8104

Epoch 00012: val_loss improved from 0.62003 to 0.59907, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.0_lr0.01_ra0.hdf5
Epoch 13/60
 - 4s - loss: 0.4405 - acc: 0.8546 - val_loss: 0.5680 - val_acc: 0.8195

Epoch 00013: val_loss improved from 0.59907 to 0.56797, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.0_lr0.01_ra0.hdf5
Epoch 14/60
 - 4s - loss: 0.4235 - acc: 0.8611 - val_loss: 0.6671 - val_acc: 0.7817

Epoch 00014: val_loss did not improve from 0.56797
Epoch 15/60
 - 4s - loss: 0.4104 - acc: 0.8648 - val_loss: 0.5987 - val_acc: 0.8120

Epoch 00015: val_loss did not improve fr

 - 4s - loss: 1.0299 - acc: 0.6828 - val_loss: 1.0639 - val_acc: 0.6652

Epoch 00005: val_loss improved from 1.10703 to 1.06390, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.0_lr0.01_ra-0.1.hdf5
Epoch 6/60
 - 3s - loss: 0.9687 - acc: 0.6964 - val_loss: 1.0009 - val_acc: 0.6764

Epoch 00006: val_loss improved from 1.06390 to 1.00091, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.0_lr0.01_ra-0.1.hdf5
Epoch 7/60
 - 3s - loss: 0.9137 - acc: 0.7114 - val_loss: 0.9626 - val_acc: 0.6930

Epoch 00007: val_loss improved from 1.00091 to 0.96262, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.0_lr0.01_ra-0.1.hdf5
Epoch 8/60
 - 3s - loss: 0.8707 - acc: 0.7235 - val_loss: 0.8923 - val_acc: 0.7087

Epoch 00008: val_loss improved from 0.96262 to 0.89229, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.0_lr0.01_ra-0.1.hdf5
Epoch 9/60
 - 3s - loss: 0.8373 - acc: 0.7311 - val_loss: 0.8826 - val_acc: 0.7056

Epo

Epoch 2/60
 - 4s - loss: 1.0700 - acc: 0.6725 - val_loss: 1.0051 - val_acc: 0.6848

Epoch 00002: val_loss improved from 1.20911 to 1.00507, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.0_lr0.01_ra-0.1.hdf5
Epoch 3/60
 - 4s - loss: 0.8987 - acc: 0.7158 - val_loss: 0.9856 - val_acc: 0.6948

Epoch 00003: val_loss improved from 1.00507 to 0.98565, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.0_lr0.01_ra-0.1.hdf5
Epoch 4/60
 - 4s - loss: 0.7976 - acc: 0.7427 - val_loss: 0.8321 - val_acc: 0.7305

Epoch 00004: val_loss improved from 0.98565 to 0.83213, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.0_lr0.01_ra-0.1.hdf5
Epoch 5/60
 - 4s - loss: 0.7256 - acc: 0.7665 - val_loss: 0.7725 - val_acc: 0.7511

Epoch 00005: val_loss improved from 0.83213 to 0.77249, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.0_lr0.01_ra-0.1.hdf5
Epoch 6/60
 - 4s - loss: 0.6630 - acc: 0.7828 - val_loss: 0.8212 - val_acc: 

...

Epoch 7/60
 - 5s - loss: 1.4467 - acc: 0.6371 - val_loss: 1.6608 - val_acc: 0.6209

Epoch 00007: val_loss did not improve from 1.65241
Epoch 8/60
 - 5s - loss: 1.4278 - acc: 0.6371 - val_loss: 1.5864 - val_acc: 0.6209

Epoch 00008: val_loss improved from 1.65241 to 1.58638, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.5_lr0.1_ra-0.1.hdf5
Epoch 9/60
 - 5s - loss: 1.4103 - acc: 0.6371 - val_loss: 1.5959 - val_acc: 0.6209

Epoch 00009: val_loss did not improve from 1.58638
Epoch 10/60
 - 5s - loss: 1.3897 - acc: 0.6371 - val_loss: 1.5729 - val_acc: 0.6209

Epoch 00010: val_loss improved from 1.58638 to 1.57286, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.5_lr0.1_ra-0.1.hdf5
Epoch 11/60
 - 5s - loss: 1.3637 - acc: 0.6372 - val_loss: 1.4345 - val_acc: 0.6214

Epoch 00011: val_loss improved from 1.57286 to 1.43447, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.5_lr0.1_ra-0.1.hdf5
Epoch 12/60
 - 5s - loss: 1.3294 - acc


Epoch 00057: val_loss did not improve from 1.14407
Epoch 58/60
 - 5s - loss: 1.1126 - acc: 0.6622 - val_loss: 1.2036 - val_acc: 0.6564

Epoch 00058: val_loss did not improve from 1.14407
Epoch 59/60
 - 5s - loss: 1.1152 - acc: 0.6620 - val_loss: 1.1749 - val_acc: 0.6595

Epoch 00059: val_loss did not improve from 1.14407
Epoch 60/60
 - 5s - loss: 1.1094 - acc: 0.6634 - val_loss: 1.2059 - val_acc: 0.6493

Epoch 00060: val_loss did not improve from 1.14407
Train on 51088 samples, validate on 6798 samples
Epoch 1/60
 - 9s - loss: 2.4901 - acc: 0.5622 - val_loss: 1.3459 - val_acc: 0.6368

Epoch 00001: val_loss improved from inf to 1.34593, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.5_lr0.1_ra-0.5.hdf5
Epoch 2/60
 - 5s - loss: 1.3512 - acc: 0.6330 - val_loss: 1.1072 - val_acc: 0.6608

Epoch 00002: val_loss improved from 1.34593 to 1.10717, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.5_lr0.1_ra-0.5.hdf5
Epoch 3/60
 - 5s - loss: 1.2036 - a

 - 5s - loss: 0.6146 - acc: 0.8022 - val_loss: 0.4694 - val_acc: 0.8585

Epoch 00041: val_loss did not improve from 0.45381
Epoch 42/60
 - 5s - loss: 0.6165 - acc: 0.8032 - val_loss: 0.4670 - val_acc: 0.8632

Epoch 00042: val_loss did not improve from 0.45381
Epoch 43/60
 - 5s - loss: 0.6082 - acc: 0.8048 - val_loss: 0.4447 - val_acc: 0.8592

Epoch 00043: val_loss improved from 0.45381 to 0.44465, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.5_lr0.1_ra-0.5.hdf5
Epoch 44/60
 - 5s - loss: 0.6101 - acc: 0.8049 - val_loss: 0.4539 - val_acc: 0.8557

Epoch 00044: val_loss did not improve from 0.44465
Epoch 45/60
 - 5s - loss: 0.6094 - acc: 0.8047 - val_loss: 0.4611 - val_acc: 0.8495

Epoch 00045: val_loss did not improve from 0.44465
Epoch 46/60
 - 5s - loss: 0.6058 - acc: 0.8061 - val_loss: 0.4493 - val_acc: 0.8611

Epoch 00046: val_loss did not improve from 0.44465
Epoch 47/60
 - 5s - loss: 0.6052 - acc: 0.8053 - val_loss: 0.4388 - val_acc: 0.8605

Epoch 00047: val

Epoch 26/60
 - 4s - loss: 1.3511 - acc: 0.6371 - val_loss: 1.3841 - val_acc: 0.6209

Epoch 00026: val_loss did not improve from 1.37158
Epoch 27/60
 - 4s - loss: 1.3429 - acc: 0.6371 - val_loss: 1.3998 - val_acc: 0.6209

Epoch 00027: val_loss did not improve from 1.37158
Epoch 28/60
 - 4s - loss: 1.3434 - acc: 0.6371 - val_loss: 1.3647 - val_acc: 0.6209

Epoch 00028: val_loss improved from 1.37158 to 1.36468, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.5_lr0.1_ra-0.5.hdf5
Epoch 29/60
 - 4s - loss: 1.3373 - acc: 0.6371 - val_loss: 1.4250 - val_acc: 0.6209

Epoch 00029: val_loss did not improve from 1.36468
Epoch 30/60
 - 4s - loss: 1.3349 - acc: 0.6371 - val_loss: 1.4318 - val_acc: 0.6209

Epoch 00030: val_loss did not improve from 1.36468
Epoch 31/60
 - 4s - loss: 1.3371 - acc: 0.6371 - val_loss: 1.3861 - val_acc: 0.6209

Epoch 00031: val_loss did not improve from 1.36468
Epoch 32/60
 - 4s - loss: 1.3322 - acc: 0.6371 - val_loss: 1.3591 - val_acc: 0.6209

Epoc

Epoch 25/60
 - 6s - loss: 1.1932 - acc: 0.6470 - val_loss: 1.2828 - val_acc: 0.6300

Epoch 00025: val_loss did not improve from 1.26851
Epoch 26/60
 - 6s - loss: 1.1885 - acc: 0.6496 - val_loss: 1.3249 - val_acc: 0.6267

Epoch 00026: val_loss did not improve from 1.26851
Epoch 27/60
 - 6s - loss: 1.1850 - acc: 0.6508 - val_loss: 1.2469 - val_acc: 0.6312

Epoch 00027: val_loss improved from 1.26851 to 1.24695, saving model to C:\Users\Tyler\Desktop\Capstone WIP\models\model3_dr0.5_lr0.1_ra-0.5.hdf5
Epoch 28/60
 - 6s - loss: 1.1834 - acc: 0.6496 - val_loss: 1.2504 - val_acc: 0.6277

Epoch 00028: val_loss did not improve from 1.24695
Epoch 29/60
 - 6s - loss: 1.1734 - acc: 0.6515 - val_loss: 1.3093 - val_acc: 0.6339

Epoch 00029: val_loss did not improve from 1.24695
Epoch 30/60
 - 6s - loss: 1.1756 - acc: 0.6511 - val_loss: 1.2695 - val_acc: 0.6361

Epoch 00030: val_loss did not improve from 1.24695
Epoch 31/60
 - 6s - loss: 1.1690 - acc: 0.6525 - val_loss: 1.2518 - val_acc: 0.6405

Epoc

#### 7. Model Evaluation and Selection

In [5]:
def model_testing(filename, x_testing, y_testing):
    # load model with minimal validation log loss
    model = load_model(filename)
    
    # score model against testing data
    score = model.evaluate(x_testing, y_testing, verbose=1)
    print('Filename: ' + str(filename)+ ' | Test accuracy:', score[1])

# retrieve filepaths of all models
dir_path = os.path.join(os.path.dirname(os.path.abspath('__file__')),'models')
models = glob.glob(os.path.join(dir_path, '*'))

# load testing data into memory
x_testing = np.load('data/x_testing.npy')
y_testing = np.load('data/y_testing.npy')

# testing each model against the testing data
for model in models:
    test = model_testing(model, x_testing, y_testing)

6835/6835 [==============================] - 1s 77us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.01_ra-0.1.hdf5 | Test accuracy: 0.8343818580397918
6835/6835 [==============================] - 1s 82us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.01_ra-0.5.hdf5 | Test accuracy: 0.8411119238773923
6835/6835 [==============================] - 1s 84us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.01_ra0.hdf5 | Test accuracy: 0.8460863204096561
6835/6835 [==============================] - 1s 86us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.05_ra-0.1.hdf5 | Test accuracy: 0.8495976591685783
6835/6835 [==============================] - 1s 93us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model1_dr0.0_lr0.05_ra-0.5.hdf5 | Test accuracy: 0.8466715435870128
6835/6835 [==============================] - 1s 92us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\mode

6835/6835 [==============================] - 2s 227us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.5_lr0.01_ra-0.5.hdf5 | Test accuracy: 0.6244330651060717
6835/6835 [==============================] - 2s 239us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.5_lr0.01_ra0.hdf5 | Test accuracy: 0.6244330651060717
6835/6835 [==============================] - 2s 225us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.5_lr0.05_ra-0.1.hdf5 | Test accuracy: 0.6244330651060717
6835/6835 [==============================] - 2s 246us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.5_lr0.05_ra-0.5.hdf5 | Test accuracy: 0.6244330651060717
6835/6835 [==============================] - 2s 237us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\models\model2_dr0.5_lr0.05_ra0.hdf5 | Test accuracy: 0.6244330651060717
6835/6835 [==============================] - 2s 234us/step
Filename: C:\Users\Tyler\Desktop\Capstone WIP\m

Below is my test data sorted by model architecture, then by dropout rate, by learning rate, and by ReLU alpha.

Model # | Dropout Rate | Learning Rate | ReLU Alpha | Test Accuracy Score
---|---|---|---|---
1 | 0 | 0.01 | 0 | 84.61%
1 | 0 | 0.01 | 0.1 | 83.44%
1 | 0 | 0.01 | 0.5 | 84.11%
1 | 0 | 0.05 | 0 | 83.04%
1 | 0 | 0.05 | 0.1 | 84.96%
1 | 0 | 0.05 | 0.5 | 84.67%
1 | 0 | 0.1 | 0 | 84.64%
1 | 0 | 0.1 | 0.1 | 82.93%
1 | 0 | 0.1 | 0.5 | 84.23%
1 | 0.25 | 0.01 | 0 | 87.94%
1 | 0.25 | 0.01 | 0.1 | 87.21%
1 | 0.25 | 0.01 | 0.5 | 87.96%
1 | 0.25 | 0.05 | 0 | 88.21%
1 | 0.25 | 0.05 | 0.1 | 87.27%
1 | 0.25 | 0.05 | 0.5 | 87.56%
1 | 0.25 | 0.1 | 0 | 87.70%
1 | 0.25 | 0.1 | 0.1 | 88.21%
1 | 0.25 | 0.1 | 0.5 | 86.01%
1 | 0.5 | 0.01 | 0 | 84.73%
1 | 0.5 | 0.01 | 0.1 | 84.80%
1 | 0.5 | 0.01 | 0.5 | 84.71%
1 | 0.5 | 0.05 | 0 | 85.41%
1 | 0.5 | 0.05 | 0.1 | 84.51%
1 | 0.5 | 0.05 | 0.5 | 85.30%
1 | 0.5 | 0.1 | 0 | 85.21%
1 | 0.5 | 0.1 | 0.1 | 84.65%
1 | 0.5 | 0.1 | 0.5 | 84.77%
2 | 0 | 0.01 | 0 | 82.27%
2 | 0 | 0.01 | 0.1 | 80.01%
2 | 0 | 0.01 | 0.5 | 82.11%
2 | 0 | 0.05 | 0 | 82.11%
2 | 0 | 0.05 | 0.1 | 81.95%
2 | 0 | 0.05 | 0.5 | 81.49%
2 | 0 | 0.1 | 0 | 82.21%
2 | 0 | 0.1 | 0.1 | 82.44%
2 | 0 | 0.1 | 0.5 | 81.07%
2 | 0.25 | 0.01 | 0 | 71.31%
2 | 0.25 | 0.01 | 0.1 | 71.02%
2 | 0.25 | 0.01 | 0.5 | 72.28%
2 | 0.25 | 0.05 | 0 | 72.32%
2 | 0.25 | 0.05 | 0.1 | 74.48%
2 | 0.25 | 0.05 | 0.5 | 73.12%
2 | 0.25 | 0.1 | 0 | 72.11%
2 | 0.25 | 0.1 | 0.1 | 70.04%
2 | 0.25 | 0.1 | 0.5 | 74.92%
2 | 0.5 | 0.01 | 0 | 62.44%
2 | 0.5 | 0.01 | 0.1 | 62.44%
2 | 0.5 | 0.01 | 0.5 | 62.44%
2 | 0.5 | 0.05 | 0 | 62.44%
2 | 0.5 | 0.05 | 0.1 | 62.44%
2 | 0.5 | 0.05 | 0.5 | 62.44%
2 | 0.5 | 0.1 | 0 | 62.44%
2 | 0.5 | 0.1 | 0.1 | 62.44%
2 | 0.5 | 0.1 | 0.5 | 62.44%
3 | 0 | 0.01 | 0 | 83.48%
3 | 0 | 0.01 | 0.1 | 83.61%
3 | 0 | 0.01 | 0.5 | 83.61%
3 | 0 | 0.05 | 0 | 82.08%
3 | 0 | 0.05 | 0.1 | 82.79%
3 | 0 | 0.05 | 0.5 | 82.38%
3 | 0 | 0.1 | 0 | 82.77%
3 | 0 | 0.1 | 0.1 | 82.15%
3 | 0 | 0.1 | 0.5 | 83.44%
3 | 0.25 | 0.01 | 0 | 83.38%
3 | 0.25 | 0.01 | 0.1 | 83.19%
3 | 0.25 | 0.01 | 0.5 | 83.44%
3 | 0.25 | 0.05 | 0 | 82.84%
3 | 0.25 | 0.05 | 0.1 | 83.79%
3 | 0.25 | 0.05 | 0.5 | 83.50%
3 | 0.25 | 0.1 | 0 | 82.47%
3 | 0.25 | 0.1 | 0.1 | 82.15%
3 | 0.25 | 0.1 | 0.5 | 82.91%
3 | 0.5 | 0.01 | 0 | 64.89%
3 | 0.5 | 0.01 | 0.1 | 66.99%
3 | 0.5 | 0.01 | 0.5 | 66.19%
3 | 0.5 | 0.05 | 0 | 65.57%
3 | 0.5 | 0.05 | 0.1 | 67.17%
3 | 0.5 | 0.05 | 0.5 | 65.31%
3 | 0.5 | 0.1 | 0 | 65.75%
3 | 0.5 | 0.1 | 0.1 | 65.85%
3 | 0.5 | 0.1 | 0.5 | 66.47%


Based on the test data above, two models did equally best in categorizing the test data and beat my 80.0% accuracy objective with 82.2%:
1. Model 1 architecture with a dropout rate of 0.25, learning rate of 0.05, and ReLU Alpha of 0.0
2. Model 1 architecture with a dropout rate of 0.25, learning rate of 0.1, and ReLU Alpha of 0.1

Upholding the concept of the simplier the better, I select the first model since it has a more conservative learning rate and its kernel is a simplier non-leaky ReLU. 

A few observations from my test results:
- Model architecture 1 was vastly superior to the other two models with all scores above the average of the testing data. This suggests to me that a greater convolution window, stride, and number of filters may render direct improvement in classification accuracy in the model
- A dropout rate of 0.25 was better than both 0 and 0.5 which isn't surprising considering that a rate of 0 leads to overfitting of test data and a substantial dropout rate will impede training as backpropogated calculations will fail to persist. This suggests an appropriate range of dropout rate should be narrower and skewed toward lower values greater than 0.
- In general, a learning rate of 0.05 tested better than rates at 0 and 0.1, suggesting that models tested in a narrower range including 0.05 may train and test better. 


#### Final Algorithm

Below is the final algorithm, using both the mfcc_conversion utility to convert audiofiles into MFCC inputs, and the selected neural network to understand the speech command. I demonstrate the algorithm's ability on some sample files in the data. 

In [7]:
def algo(audio_path, filename):
    print("My prediction for "+filename+" is...")
    model_path = os.path.join(os.path.dirname(os.path.abspath('__file__')),'models','model1_dr0.25_lr0.1_ra0.hdf5')
    model = load_model(model_path)
    mfccs = mfcc_conversion(audio_path, filename).reshape((1,79,12))
    prediction = np.argmax(model.predict(mfccs))
    if prediction == 1:
        prediction = "yes"
    elif prediction == 2:
        prediction = "no"
    elif prediction == 3:
        prediction = "up"
    elif prediction == 4:
        prediction = "down"
    elif prediction == 5:
        prediction = "left"
    elif prediction == 6:
        prediction = "right"
    elif prediction == 7:
        prediction = "on"
    elif prediction == 8:
        prediction = "off"
    elif prediction == 9:
        prediction = "stop"
    elif prediction == 10:
        prediction = "go"
    else:
        prediction = "UNKNOWN"
    print(prediction)

# using my own data for demonstration and review
dir_path = os.path.join(os.path.dirname(os.path.abspath('__file__')),'tyler_audio')
sample_files = ['bed.wav', 'bird.wav', 'cat.wav', 'dog.wav', 'down.wav', 'eight.wav', 'five.wav','four.wav','go.wav',
                'house.wav','happy.wav','left.wav','marvin.wav','nine.wav','no.wav','off.wav','on.wav','one.wav',
                'right.wav','seven.wav','sheila.wav','six.wav','stop.wav','three.wav','tree.wav','two.wav',
                'up.wav','wow.wav','yes.wav','zero.wav']

for sample_file in sample_files:
    algo(dir_path,sample_file)

My prediction for bed.wav is...


C:\Users\Tyler\Anaconda3\envs\tfspeech\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


UNKNOWN
My prediction for bird.wav is...
UNKNOWN
My prediction for cat.wav is...
UNKNOWN
My prediction for dog.wav is...
UNKNOWN
My prediction for down.wav is...
down
My prediction for eight.wav is...
UNKNOWN
My prediction for five.wav is...
UNKNOWN
My prediction for four.wav is...
UNKNOWN
My prediction for go.wav is...
go
My prediction for house.wav is...
UNKNOWN
My prediction for happy.wav is...
UNKNOWN
My prediction for left.wav is...
UNKNOWN
My prediction for marvin.wav is...
UNKNOWN
My prediction for nine.wav is...
UNKNOWN
My prediction for no.wav is...
no
My prediction for off.wav is...
off
My prediction for on.wav is...
on
My prediction for one.wav is...
UNKNOWN
My prediction for right.wav is...
right
My prediction for seven.wav is...
UNKNOWN
My prediction for sheila.wav is...
UNKNOWN
My prediction for six.wav is...
UNKNOWN
My prediction for stop.wav is...
stop
My prediction for three.wav is...
UNKNOWN
My prediction for tree.wav is...
UNKNOWN
My prediction for two.wav is...
UNKN

#### Summary, Conclusion & Final Thoughts

To summarize, the purpose of this project was to build an algorithm that could understand a small library of 10 simple commands while being able to ignore or not interpret unknown commands. Using Mel-Frequency Capestral Coefficients as the medium with which to regularize and convey communicated information, I trained a 1-dimensional convolutional neural network, using RMSprop to minimize the categorical crossentropy log loss, to process and interpret the information in order to make a prediction as to the intended command.

3 different network architectures were designed and trained. Hyperparameters - dropbout rates, learning rates, and the leaky rectified linear unit alpha - were toggled for each of the 3 architectures resulting in 81 trained models. All 81 models were tested and the algorithm that scored highest - 88.2% - was selected. This score was higher than the 80.0% I originally hoped for.

In conclusion, I want to leave with a few ideas I would like to incorporate into this project - or a similar project - in a future date:
- Using recurrent networks, or long-short term memory networks, to process the MFCCs. While I used convolutional neural networks to achieve my goal, perhaps other network architectures would be more applicable such as RNNs and LSTMs which are widely used in signal processing and speech recognition. Perhaps I'll repeat my procedures here with such architectures to see if I can improve upon my current accuracy. 
- Augment and distort signals for testing. While I originally intended to add some random digital distortion to my input data in order to account for more realistic use cases and to aid against overfitting the models to only the noiseless cases, I ultimately decided this wasn't necessary for my purposes here. However, should I decide to incorporate distinctions between the commands, silence, and noise, then I can script the necessary code to do so.
- Reduce the sample rate of inputs to see if I can maintain accuracy with less information. While I trained the model with audio files decoded at 16000Mbps, perhaps I can achieve the same accuracy - or perhaps improve upon - with audio sampled at a lower bit rate - maybe 8000Mbps, for example. If successful, this would allow my algorithm to be used with electronics or networks with a more limited capacity.


#### Works Cited

Alameda Internet Marketing. (2016, 07 06). 39 Experts Share Their Top 3 SEO Trends for 2017 and Beyond. Retrieved from Alameda Internet Marketing: https://alamedaim.com/seo-trends/

Practical Cryptography. (2018, 06 01). Mel Frequency Cepstral Coefficient (MFCC) tutorial. (http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/

Sterling, G. (2016, 05 18). Google says 20 percent of mobile queries are voice searches. Retrieved from Search Engine Land: https://searchengineland.com/google-reveals-20-percent-queries-voice-queries-249917

Young, W. (2016, 06 20). The voice search explosion and how it will change local search. Retrieved from Search Engine Land: https://searchengineland.com/voice-search-explosion-will-change-local-search-251776